Creazione Dimensione customer

In [8]:
%%sql
create or replace table gold_DimCustomer
using delta  PARTITIONED BY (CustomerID)
SELECT CAST(ROW_NUMBER() OVER(ORDER BY C.CustomerID) AS INT) AS CustomerKey,
        CAST(C.CustomerID AS INT) AS CustomerID,
        C.CustomerName,
        CC.CustomerCategoryName,
        BG.BuyingGroupName,
        DM.DeliveryMethodName,
        DC.CityName AS DeliveryCityName,
        DSP.StateProvinceName AS DeliveryStateProvinceName,
        DSP.SalesTerritory AS DeliverySalesTerritory,
        DCO.CountryName AS DeliveryCountry,
        DCO.Continent AS DeliveryContinent,
        DCO.Region AS DeliveryRegion,
        DCO.Subregion AS DeliverySubregion,
        CAST('2013-01-01' AS DATE) AS ValidFromDate
FROM silver_Customers C
LEFT JOIN silver_CustomerCategories CC On CC.CustomerCategoryID = C.CustomerCategoryID
LEFT JOIN silver_Cities  DC ON DC.CityID = C.DeliveryCityID
LEFT JOIN silver_StateProvinces  DSP ON DSP.StateProvinceID = DC.StateProvinceID
LEFT JOIN silver_Countries DCO ON DCO.CountryID = DSP.CountryID
LEFT JOIN silver_SalesBuyingGroups BG ON BG.BuyingGroupID = C.BuyingGroupID
LEFT JOIN silver_DeliveryMethods DM ON DM.DeliveryMethodID = C.DeliveryMethodID



StatementMeta(, 5c423571-5039-4e96-b195-79e66192f09c, 9, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

Creazione dimensione Stock

In [9]:
%%sql
create or replace table gold_DimStockItem
using delta  PARTITIONED BY (StockItemID) 
SELECT CAST(ROW_NUMBER() OVER(ORDER BY SI.StockItemID) AS SMALLINT) AS StockItemKey,
CAST(SI.StockItemID AS SMALLINT) AS StockItemID,
SI.StockItemName,
SI.LeadTimeDays,
C.ColorName,
OP.PackageTypeName AS OuterPackageTypeName,
CAST('2013-01-01' AS DATE) AS ValidFromDate,
CAST(SI.SupplierID AS TINYINT) AS SupplierID
FROM silver_StockItems SI
LEFT JOIN silver_Colors C ON C.ColorID = SI.ColorID
LEFT JOIN silver_PackageTypes OP ON OP.PackageTypeID = SI.OuterPackageID

StatementMeta(, 5c423571-5039-4e96-b195-79e66192f09c, 10, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
create or replace table gold_DimSupplier
using delta PARTITIONED BY (SupplierKey) 
SELECT CAST(ROW_NUMBER() OVER(ORDER BY S.SupplierID) AS TINYINT) AS SupplierKey,
CAST(S.SupplierID AS TINYINT) AS SupplierID,
S.SupplierName,
SC.SupplierCategoryName,
CAST('2013-01-01' AS DATE) AS ValidFromDate
FROM silver_Suppliers S
LEFT JOIN silver_SupplierCategories SC ON SC.SupplierCategoryID = S.SupplierCategoryID

StatementMeta(, 5c423571-5039-4e96-b195-79e66192f09c, 12, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

Creazione dimensione Date

In [12]:


%%sql

create or replace table gold_DimDate
using delta PARTITIONED BY (DateKey)
SELECT CAST(DateKey AS INT) AS DateKey,
        CAST(Date AS DATE) AS Date,
        CAST(Day AS TINYINT) AS Day,
        CAST(WeekDay AS TINYINT) AS WeekDay,
        WeekDayName,
        CAST(Month AS TINYINT) AS Month,
        MonthName,
        CAST(Quarter AS TINYINT) AS Quarter,
        CAST(Year AS SMALLINT) AS Year
        from silver_date;

StatementMeta(, 5c423571-5039-4e96-b195-79e66192f09c, 13, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

Creazione Tabella dei fatti

In [13]:
%%sql
create or replace table gold_FactSales
using delta PARTITIONED BY (OrderDateKey)
SELECT  
  --Surrogate Keys 
    DC.CustomerKey,
    year(SO.OrderDate) * 10000 + month(SO.OrderDate) * 100 + day(SO.OrderDate) as OrderDateKey,
    --CAST(DATE_FORMAT(SO.OrderDate,'yyyyMMdd') as INT) as OrderDateKey,
    --SO.OrderDate,
    SI.StockItemKey,
    DS.SupplierKey,
    --Degenerate Dimensions
    CAST(SO.OrderID AS INT) AS OrderID,
    CAST(SOL.OrderLineID AS INT) AS OrderLineID,  
    --Measure
    CAST(SOL.Quantity AS INT) AS SalesOrderQuantity, 
    CAST(SOL.UnitPrice AS DECIMAL(18,2)) AS SalesOrderUnitPrice
FROM silver_SalesOrderLine SOL
INNER JOIN silver_SalesOrder SO ON SOL.OrderID = SO.OrderID
INNER JOIN gold_DimCustomer DC ON DC.CustomerID = SO.CustomerID
INNER JOIN gold_DimStockItem SI ON SI.StockItemID = SOL.StockItemID
INNER JOIN gold_DimSupplier DS ON DS.SupplierID = SI.SupplierID;

StatementMeta(, 5c423571-5039-4e96-b195-79e66192f09c, 14, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>